# 파이토치 신경망 구현하기

## 필요한 라이브러리 임포트

In [2]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using {device} device")

Using cpu device


## 신경망 정의하기

In [17]:
# nn.Module로부터 딥러닝에 필요한 상당부분의 기능을 상속받기 때문에 
# NeuralNetwork class를 정의할 때 중복되는 코드를 작성할 필요가 없다.
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__() # 상속 받은 모듈의 init을 수행
        # 커스터마이징할 내용을 정의
        self.linear_relu_stack = nn.Sequential(
            nn.Flatten(),  # 28*28 이미지를 784로 변환
            nn.Linear(784, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.Softmax(dim=1)
        )
        
    def forward(self, x):
        pred_probab = self.linear_relu_stack(x) # 포워드 패스
        return pred_probab
    
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (linear_relu_stack): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=784, out_features=512, bias=True)
    (2): ReLU()
    (3): Linear(in_features=512, out_features=512, bias=True)
    (4): ReLU()
    (5): Linear(in_features=512, out_features=10, bias=True)
    (6): Softmax(dim=1)
  )
)


## 정의된 모델로 연산하기

In [18]:
X = torch.rand(1, 28, 28, device=device)
pred_probab = model(X)
y_pred = pred_probab.argmax(1)
print(f'softmax result: {pred_probab}')
print(f"Predicted class: {y_pred}")

softmax result: tensor([[0.0941, 0.1048, 0.1013, 0.0966, 0.1118, 0.0861, 0.1033, 0.0935, 0.1065,
         0.1020]], grad_fn=<SoftmaxBackward0>)
Predicted class: tensor([4])


## 각 레이어의 파라미터 확인

파이토치에 설정된 각 레이어와 파라미터 정보를 for문으로 확인

In [20]:
print(f"모델 구조: {model}\n\n")

for name, param in model.named_parameters():
    print(f"레이어: {name} | 사이즈: {param.size()} | 값: {param[:2]} \n")

모델 구조: NeuralNetwork(
  (linear_relu_stack): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=784, out_features=512, bias=True)
    (2): ReLU()
    (3): Linear(in_features=512, out_features=512, bias=True)
    (4): ReLU()
    (5): Linear(in_features=512, out_features=10, bias=True)
    (6): Softmax(dim=1)
  )
)


레이어: linear_relu_stack.1.weight | 사이즈: torch.Size([512, 784]) | 값: tensor([[ 0.0019,  0.0114,  0.0165,  ...,  0.0062, -0.0329,  0.0032],
        [ 0.0231, -0.0262,  0.0025,  ..., -0.0341,  0.0324, -0.0098]],
       grad_fn=<SliceBackward0>) 

레이어: linear_relu_stack.1.bias | 사이즈: torch.Size([512]) | 값: tensor([0.0049, 0.0109], grad_fn=<SliceBackward0>) 

레이어: linear_relu_stack.3.weight | 사이즈: torch.Size([512, 512]) | 값: tensor([[-0.0355, -0.0406,  0.0153,  ...,  0.0055, -0.0229,  0.0009],
        [ 0.0391, -0.0169,  0.0360,  ...,  0.0361,  0.0010,  0.0404]],
       grad_fn=<SliceBackward0>) 

레이어: linear_relu_stack.3.bias | 사이즈: torch.Size([512]